In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/mpr_sensors.csv")
df = df.rename(columns={'ReadDatetime':'datetime'})

In [3]:
display(df)

,MPR,datetime,kWh,ReadType
0,1.200000e+12,04/02/2023 00:00,26.6,Actual
1,1.200000e+12,04/02/2023 00:30,26.4,Actual
2,1.200000e+12,04/02/2023 01:00,25.8,Actual
3,1.200000e+12,04/02/2023 01:30,24.9,Actual
4,1.200000e+12,04/02/2023 02:00,25.9,Actual
...,...,...,...,...
187,1.200060e+12,05/02/2023 21:30,0.0,Actual
188,1.200060e+12,05/02/2023 22:00,0.0,Actual
189,1.200060e+12,05/02/2023 22:30,0.0,Actual
190,1.200060e+12,05/02/2023 23:00,0.0,Actual


In [4]:
#converting time column to datetime and making time as index column
df['datetime'] = pd.to_datetime(df['datetime'], format = '%d/%m/%Y %H:%M')

In [5]:
df = df.set_index("datetime")
# Find the first and last dates in the index of the DataFrame
first_date = df.index.min()
last_date = df.index.max()
# Print the results
print("The first date in the time series is {}.".format(first_date))
print("The last date in the time series is {}.".format(last_date))

The first date in the time series is 2023-02-04 00:00:00.
The last date in the time series is 2023-02-05 23:30:00.


In [6]:
#aggregate to hourly
df_mpr_hourly = df.resample('H').mean()

## Read weather data for these dates

In [7]:
# Import Meteostat library and dependencies
from datetime import datetime
from meteostat import Hourly

# Set time period
start = datetime(2023, 2, 4)
end = datetime(2023, 2, 5, 23, 59)

# Get hourly data
data = Hourly('03779', start, end)
df_weather = data.fetch()

## Merging

In [8]:
# Merge the DataFrames based on their index columns
merged_df = pd.merge(df_mpr_hourly, df_weather, left_index=True, right_index=True)

In [9]:
from ydata_profiling import ProfileReport
profile = ProfileReport(merged_df, title="Profiling Report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
merged_df['date'] = merged_df.index.date
merged_df['time'] = merged_df.index.time

In [14]:
merged_df.to_csv('../data/merged_weather_mpr_data.csv', sep =',', header = True)